In [ ]:
import sys
import numpy as np
from importlib import reload
import networkx as nx

sys.path.insert(0, '/home/wwj/Repo/playgrounds/NeuralForceField/')

import torch
import torch.nn as nn
import copy
import torch.nn.functional as F

from nff.nn.layers import Dense, GaussianSmearing
from nff.nn.modules import GraphDis, SchNetConv, BondEnergyModule, SchNetEdgeUpdate, NodeMultiTaskReadOut
from nff.nn.activations import shifted_softplus
from nff.nn.graphop import batch_and_sum, get_atoms_inside_cell
from nff.nn.utils import get_default_readout

from torch.utils.data import DataLoader
import nff.data as d
import pickle

from nff.data import Dataset, split_train_validation_test, collate_dicts

from nff.io.ase import * 

from shutil import rmtreeb

In [ ]:
# initialize model parameters 

modelparams = dict()
modelparams['n_atom_basis'] = 128
modelparams['n_filters'] = 128
modelparams['n_gaussians'] = 32
modelparams['mol_n_convolutions'] = 3
modelparams['sys_n_convolutions'] = 3
modelparams['mol_cutoff'] = 4.0
modelparams['sys_cutoff'] = 5.0
# modelparams["V_ex_power"] = 12
# modelparams["V_ex_sigma"] = 4.0

In [ ]:
from torch.optim import Adam
from nff.data import Dataset, split_train_validation_test, collate_dicts, sparsify_tensor
from nff.train import Trainer, get_trainer, get_model, loss, hooks, metrics, evaluate
from nff.nn.models.hybridgraph import HyBridGraphConvb

In [ ]:
props = pickle.load( open( "./data/ethane_data.pkl", "rb" ) )
props['offsets'] = [sparsify_tensor(offset.matmul(torch.Tensor(props["cell"][i]))) for i, offset in enumerate(props['offsets'])]
dataset = d.Dataset(props.copy(), units='kcal/mol')

train, val, test = split_train_validation_test(dataset, val_size=0.1, test_size=0.1)

train_loader = DataLoader(train, batch_size=1, collate_fn=collate_dicts)
val_loader = DataLoader(val, batch_size=1, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=1, collate_fn=collate_dicts)b

In [1]:
# Training 

In [ ]:
model = HyBridGraphConv(modelparams)

loss_fn = loss.build_mse_loss(loss_coef={'energy_grad': 1})

model = HyBridGraphConv(modelparams)


trainable_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = Adam(trainable_params, lr=3e-4)


train_metrics = [
    metrics.MeanAbsoluteError('energy_grad')
]

from shutil import rmtree
import os
OUTDIR = "./CG_test1"
train_hooks = [
    hooks.MaxEpochHook(100),
    hooks.CSVHook(
        OUTDIR,
        metrics=train_metrics,
    ),
    hooks.PrintingHook(
        OUTDIR,
        metrics=train_metrics,
        separator = ' | ',
        time_strf='%M:%S'
    ),
    hooks.ReduceLROnPlateauHook(
        optimizer=optimizer,
        patience=30,
        factor=0.5,
        min_lr=1e-7,
        window_length=1,
        stop_after_min=True
    )
]

if os.path.exists(OUTDIR):
    rmtree(OUTDIR)b

In [ ]:
T = Trainer(
    model_path=OUTDIR,
    model=model,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_loader=train_loader,
    validation_loader=val_loader,
    checkpoint_interval=1,
    hooks=train_hooks
)

In [ ]:
T.train(device=0, n_epochs=15)

In [ ]:
# Dynamics 

In [ ]:
from ase import Atoms
from ase.neighborlist import neighbor_list
from nff.data.sparse import sparsify_array

DEFAULT_CUTOFF = 5.0

system_prop = {key: val[0] for key, val in props.items()}
system_prop['atoms_cutoff'] = 4.0
system_prop['system_cutoff'] = 5.0b

In [ ]:
from nff.io.ase import BulkPhaseMaterials

bulk = BulkPhaseMaterials(numbers=[1, 1] * 64, 
                               positions=props['nxyz'][0][:, 1:4],
                               cell=props['cell'][0],
                               pbc=True,
                               props=system_prop)
bulk.set_masses([15.035, 15.035] * 64) # mass of cg atoms 
bulk.update_nbr_list()

In [ ]:
from nff.io import NeuralFF
from nff.md.nve import * 
calc = NeuralFF(model=model, device=0)
bulk.set_calculator(calc)
nve = Dynamics(bulk, DEFAULTNVEPARAMS)

In [ ]:
nve.run()

# save frames as xyz 
nve.save_as_xyz()